- 国内外优化求解器 https://zhuanlan.zhihu.com/p/575953675
- python运筹优化实践1-8 https://www.zhihu.com/column/c_1060904037507006464
    - Cplex for python使用简介 https://zhuanlan.zhihu.com/p/124422566
- 案例广场案例 https://opt.aliyun.com/platform/case
- 线性规划-专题多篇 https://help.aliyun.com/document_detail/298212.html

```shell
pip install docplex
pip install cplex
```

In [2]:
import random

import docplex.mp.model as cpx
import pandas as pd

iIdx = range(1, 10 + 1)
jIdx = range(1, 5 + 1)
xMin = {(i, j): random.randint(0, 10) for i in iIdx for j in jIdx}
xMax = {(i, j): random.randint(10, 20) for i in iIdx for j in jIdx}
cost = {(i, j): random.normalvariate(0, 1) for i in iIdx for j in jIdx}
varA = {(i, j): random.normalvariate(0, 5) for i in iIdx for j in jIdx}
varB = {j: random.randint(0, 30) for j in jIdx}

model = cpx.Model(name="MIP Model")

# if x is Continuous
xArr = {
    (i, j): model.continuous_var(
        lb=xMin[i, j], ub=xMax[i, j], name="x_{0}_{1}".format(i, j)
    )
    for i in iIdx
    for j in jIdx
}

# <= constraints，小于等于
constraints = {
    j: model.add_constraint(
        ct=model.sum(varA[i, j] * xArr[i, j] for i in iIdx) <= varB[j],
        ctname="constraint_{0}".format(j),
    )
    for j in jIdx
}

objective = model.sum(xArr[i, j] * cost[i, j] for i in iIdx for j in jIdx)

# for maximization
model.maximize(objective)

# solving with local cplex
model.solve()

# output
df = pd.DataFrame.from_dict(xArr, orient="index", columns=["variable_object"])
df.index = pd.MultiIndex.from_tuples(df.index, names=["column_i", "column_j"])
df.reset_index(inplace=True)

# CPLEX
df["solution_value"] = df["variable_object"].apply(lambda item: item.solution_value)

df.drop(columns=["variable_object"], inplace=True)
# df.to_csv("./optimization_solution.csv")

df

,column_i,column_j,solution_value
0,1,1,18.000000
1,1,2,9.000000
2,1,3,3.000000
3,1,4,7.000000
4,1,5,20.000000
5,2,1,10.000000
6,2,2,13.000000
7,2,3,8.000000
8,2,4,4.000000
9,2,5,19.000000
